# Project: Facial emotion recognition
## MTH767P - Group 4

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from pathlib import Path
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from utils import *

## Data loading and pre-processing 

#### Read data

In [3]:
# define dictionary for emotions
emotion_dict = {0: 'Angry', 
                1: 'Disgust', 
                2: 'Fear',
                3: 'Happy', 
                4: 'Sad',
                5: 'Surprise',
                6: 'Neutral'}

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
dataset = EmotionsDataset(root='./dataset', fname='icml_face_data.csv', transform=transform)

#### Visualize data


In [6]:
# generate sample of images
#fig, ax = plt.subplots(5, len(emotion_dict), figsize=(8, 4.5))

#for i, em in emotion_dict.items():
    #emotion = dataset.data[dataset.targets==i][:5]
    # set title
    #ax[0, i].set_title(em)
    #for j in range(emotion.shape[0]):      
        #im = emotion[j]
        #ax[j, i].imshow(im, cmap='gray')
        #ax[j, i].axis('off')
#plt.tight_layout();
#plt.savefig(output_path/'dataset_sample.png')

#### Split dataset

In [7]:
batch_size = 128
trainloader, testloader = dataset.split(ratio=0.8, batch_size=batch_size)

---
## Convolutional neural network setup 
### Network architecture

In [34]:
# Architecture of a simple convnet
# Specify your model by providing the list of dicts, each containing the name of the layer class (case-insensitive) 
# and the parameters necessary to initialise it. The in_channels and in_features are not necessary as they are 
# calculated automatically
# If the layer contains the activation function, give an initialised object of the relevant class

layers = [dict(ltype='conv2d', out_channels=6, kernel=7, activation=nn.ReLU(True)),
          dict(ltype='maxpool2d', kernel=2, stride=2),
          dict(ltype='conv2d', out_channels=16, kernel=5),
          dict(ltype='maxpool2d', kernel=2, stride=2, activation=nn.ReLU(True)),
          dict(ltype='linear', out_features=120, activation=nn.ReLU(True)),
          dict(ltype='linear', out_features=7)
         ]
net = ConvNet(layers)

In [41]:
# Architecture with localization network (see Minaee 2021, and code in https://pytorch.org/tutorials/intermediate/spatial_transformer_tutorial.html)

atn_layers = dict(
            attention=[dict(ltype='conv2d', out_channels=8, kernel=3),
                       dict(ltype='maxpool2d', kernel=2, stride=2, activation=nn.ReLU(True)),
                       dict(ltype='conv2d', out_channels=10, kernel=3),
                       dict(ltype='maxpool2d', kernel=2, stride=2, activation=nn.ReLU(True)),
                       dict(ltype='linear', out_features=48, activation=nn.ReLU(True)),
                       dict(ltype='linear', out_features=3*2)
                      ],
            features=[dict(ltype='conv2d', out_channels=10, kernel=3, activation=nn.ReLU(True)),
                      dict(ltype='conv2d', out_channels=10, kernel=3),
                      dict(ltype='maxpool2d', kernel=2, stride=2, activation=nn.ReLU(True)),
                      dict(ltype='conv2d', out_channels=10, kernel=3, activation=nn.ReLU(True)),
                      dict(ltype='conv2d', out_channels=10, kernel=3),
                      dict(ltype='maxpool2d', kernel=2, stride=2, activation=nn.ReLU(True)),
                      dict(ltype='dropout', p=0.5),
                      dict(ltype='linear', out_features=50),
                      dict(ltype='linear', out_features=7)
                     ])
net = AttentionalNet(atn_layers)

In [42]:
# Weight initialisation
@torch.no_grad()
def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.normal_(m.weight, 0, 0.05)
        nn.init.normal_(m.bias, 0, 0.05)
    
net = net.apply(init_weights)

In [16]:
# Display the architecture in plain text format
net

AttentionalNet(
  (localization): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 10, kernel_size=(3, 3), stride=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU(inplace=True)
  )
  (fc_loc): Sequential(
    (0): Linear(in_features=1000, out_features=48, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=48, out_features=6, bias=True)
  )
  (conv): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU(inplace=True)
    (5): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (8): 

### Loss function and optimizer

In [43]:
# Learning rate calc
stdata = (dataset.data / 255 - 0.5) / 0.5
lr = 3.9/np.linalg.norm(stdata.reshape(-1, 48*48))**2
lr = 0.005

# L2 regularisation parameter
lmda = 0.001

# Re-weighting classes for imbalance
weights = 1 / np.unique(dataset.targets[dataset.train_idxs], return_counts=True)[1]

In [44]:
criterion = nn.CrossEntropyLoss(torch.tensor(weights / np.sum(weights), dtype=torch.float))
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=lmda)  # optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = None  # optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)  # for adaptive learning rate 

---
## Training the neural network
### Hyper parameters tuning

In [325]:
# Define number of folds to split the datasets into
# Define a grid of parameter values to check (if more then one param is tested - supply list of tuples as a grid)
# Define the objective function by using cross_validate() method that will accept iterated values of object(s) using 
# the parameter(s)

from itertools import product

# Hyper parameters
lmdas = np.append(np.linspace(0, 0.01, 2), np.linspace(0.02, 0.1, 3))  # <-- regularisation parameters
lrs = pow(10, -np.linspace(1, 3, 3))  # <-- learning rates
dps = [0.3, 0.5]  # np.linspace(0.1, 0.5, 3)  # <-- dropout rates
grid = list(product(lmdas, lrs, dps))

Kfolds = 5

def eval_gridpoint(pt):
    global atn_layers, Kfolds, dataset, criterion, scheduler, emotion_dict, batch_size
    l, lr, dp = pt
    opt = optim.Adam(net.parameters(), lr=lr, weight_decay=l)
    do_update = dict(ltype='dropout', p=dp)
    new_spec = atn_layers.copy()
    new_spec['features'][-3] = do_update
    new_net = AttentionalNet(new_spec)
    return cross_validate(Kfolds, dataset, new_net, criterion, opt, scheduler, emotion_dict, batch_size) 

errors = []  # <-- in case there is an error during execution, we will still retain all previously calculated values
cur_best = (0.01, 0.001, 0.3)
bestval = grid_search(eval_gridpoint, grid, 'Regularisation, learning rate, dropout rate', out=errors)

Testing Regularisation, learning rate, dropout rate value (0.0, 0.1, 0.3)...
Fold 1:
Epoch 1


5742it [01:51, 51.73it/s]


Finished training. Avg loss: 1.9506


100%|██████████| 5742/5742 [00:16<00:00, 338.84it/s]


Fold 2:
Epoch 1


...

Fold 5:
Epoch 1


5742it [01:56, 49.29it/s]


Finished training. Avg loss: 1.9397


100%|██████████| 5741/5741 [00:16<00:00, 349.83it/s]

Done, error=0.860845


### Training

In [45]:
%%time
# Train the model by supplying the netwrok object, loss function, optimizer, learning rate adjuster (scheduler), 
# trainloader and number of times to go over the dataset.
# The scheduler is optional and can be None. It can also be a list of several schedulers

num_epochs = 30
losses = train(net, criterion, optimizer, scheduler, trainloader, num_epochs, verbose=False)

Epoch 1


225it [01:20,  2.79it/s]


Epoch 2


225it [01:20,  2.79it/s]


Epoch 3


...

Epoch 29


225it [04:01,  1.07s/it]


Epoch 30


225it [03:37,  1.03it/s]

Finished training. Avg loss: 1.8609
CPU times: user 1h 31min 51s, sys: 3min 2s, total: 1h 34min 54s
Wall time: 1h 30min 15s


---
## Testing

In [46]:
# Test the trained network on the testing dataset
# Supply the network object, loss function, testloader and dictionary of class labels
# The function will return a DataFrame with the network average loss and accuracy per class. 
# Pass this df to save_model() function in the next cell if you think it is a good model

result = check_accuracy(net, criterion, testloader, emotion_dict)

100%|██████████| 7178/7178 [00:43<00:00, 165.41it/s]

Average loss: 1.7656	Accuracy: 2272/7178 (31.7%)
Accuracy for class Angry: 13.8%
Accuracy for class Disgust: 31.2%
Accuracy for class Fear : 7.3%
Accuracy for class Happy: 61.9%
Accuracy for class Sad  : 5.7%
Accuracy for class Surprise: 70.9%
Accuracy for class Neutral: 24.6%


---
## Save and load the model

In [ ]:
# Save the current model by supplying the net object, the parameters needed to initialize it from scratch (can be None),
# the loss function, the optimizer, the number of epochs used in training, the testing results DataFrame 
# and a random batch (needed for saving in ONNX format for further visualisation)

model_name = save_model(model, criterion, optimizer, scheduler, no_epochs, result, next(iter(trainloader))[0])

In [ ]:
# Provide the name of the spec file of a model (without the extension) to load
# previously saved model as well as config with parameters used

mod_name = None # Assign the name of previously saved model
net, cfg = load_model(mod_name)